# Running Monte Carlo Simulations on an Excel Model

Here I will demonstrate using `xlwings` to drive Excel to run Monte Carlo Simulations. For this exercise, please download the Excel retirement model "Excel with Salary.xlsx" from Canvas. Then open the workbook and keep it open. Make sure you have no other workbooks open. Switch to the "Wealth" tab if it is not open.

In [ ]:
import xlwings as xw

## The Monte Carlo Setup

First let's get a connection to our Excel sheet.

In [1]:
import xlwings as xw

book = xw.Book('Dynamic Salary Retirement Model.xlsx')
sht = book.sheets['Inputs and Outputs']

### Running a Single Simulation

We want to evaluate how interest rate affects years until retirement. Let's first just try changing the interest rate and getting the years to retirement as the output.

In [2]:
sht.range('B10').value = 0.08

Now we can see that the interest rate has changed to 8% in Excel and that the years to retirement has changed to 24. But we want to get that output out of Excel as well.

In [3]:
years_to_retirement = sht.range('B18').value
years_to_retirement

24.0

Now that we have the value in Python we can analyze it in Python. Or if we want to analyze the results in Excel, we can output it back to the Excel workbook as a hard-coded value in a different cell, so that it will still be saved when the inputs change.

In [4]:
sht.range('E2').value = years_to_retirement

Now we can see the value is in Excel in the cell `E2`.

### Running Multiple Simulations

Just as we have done with pure Python Monte Carlo simulations, now we want to run this process many times. We'll use a loop over the number of iterations to do this. We will collect the results in Python and then output to Excel at the end.

First we need to be getting the interest rate randomly from a normal distribution:

In [5]:
import random

interest_mean = 0.05
interest_std = 0.03

interest_rate = random.normalvariate(interest_mean, interest_std)
interest_rate

0.0341134554522831

In [6]:
num_iter = 10

all_retirement_years = []
for i in range(num_iter):
    interest_rate = random.normalvariate(interest_mean, interest_std)
    sht.range('B10').value = interest_rate
    years_to_retirement = sht.range('B18').value
    all_retirement_years.append(years_to_retirement)
all_retirement_years

[37.0, 28.0, 26.0, 30.0, 25.0, 29.0, 38.0, 30.0, 27.0, 23.0]

Now output back to Excel. We want them in a column so we will do the list comprehension trick.

In [7]:
vertical_retirement_years = [[ret_year] for ret_year in all_retirement_years]
sht.range('E2').value = vertical_retirement_years

Now wrap this all up in a function.

In [8]:
def retirement_simulations(num_iter, interest_mean, interest_std):
    all_retirement_years = []
    for i in range(num_iter):
        interest_rate = random.normalvariate(interest_mean, interest_std)
        sht.range('B10').value = interest_rate
        years_to_retirement = sht.range('B18').value
        all_retirement_years.append(years_to_retirement)
    
    vertical_retirement_years = [[ret_year] for ret_year in all_retirement_years]
    sht.range('E2').value = vertical_retirement_years
    
    return all_retirement_years  # return it so we will also have it in Python in addition to Excel

results = retirement_simulations(1000, 0.1, 0.05)
results[:10]

[19.0, 20.0, 25.0, 24.0, 21.0, 18.0, 21.0, 22.0, 19.0, 17.0]

### Visualize and Analyze

From here, we could follow the same exact process to visualize and analyze in Python as is shown in the Monte Carlo example and practice problem. Here I will show a similar process but doing it in Excel.

#### Histogram

Select the values in Excel, then do Insert > Charts > Histogram. You can change the number of bins by going to axis formatting options.

#### Percentile Table

Excel has the `=PERCENTILE` function. Set up your table with the probabilities you want to examine, then in the first cell call the `=PERCENTILE` function on the entire data with a fixed reference, then the probability with a relative reference. Then you can drag it down to iterate through the probabilities.

### Probability of a Certain Outcome

We can recreate what we did in Python by using a simple `=IF` call. E.g. if we wanted to see the probability that years to retirement is greater than 25, we would do `=IF(E2>25,1,0)` so that we get a 1 if it is meeting the condition and zero otherwise. Then we just take an average of all the 1s and 0s.